This notebook implementation incorporates elements from [this](https://keras.io/examples/vision/autoencoder/) Keras example.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from fkan.tensorflow import FractionalJacobiNeuralBlock as fJNB
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

2024-06-12 00:03:32.354810: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def psnr(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)


def ssim(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.ssim(y_true, y_pred, max_val=max_pixel)


def preprocess(array):
    """Normalizes the supplied array and reshapes it."""
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array


def noise(array):
    """Adds random noise to each image in the supplied array."""
    noise_factor = 0.3
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)

In [3]:
# Since we only need images from the dataset to encode and decode, we
# won't use the labels.
(train_data, _), (test_data, _) = fashion_mnist.load_data()

# assume x_train is your original dataset
train_data, valid_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)
valid_data = preprocess(valid_data)

# Create a copy of the data with added noise
noisy_train_data = noise(train_data)
noisy_test_data = noise(test_data)
noisy_valid_data = noise(valid_data)

In [4]:
batch_size = 512
epochs = 20
q = 2  # See paper for the definition and role of q
trial = 1

If using a predefined Keras activation function, comment out `fJNB(q)` and uncomment `x = layers.Activation(activation=activation)(x)`, specifying your desired activation function.

In [5]:
input = layers.Input(shape=(28, 28, 1))

# Encoder
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Flatten the encoded representation
x = layers.Flatten()(x)

# Dense layer in the middle
x = layers.Dense(98)(x)

# x = layers.Activation(activation=activation)(x)
x = fJNB(q)(x)


# Reshape back to the original shape before decoder
x = layers.Reshape((7, 7, 2))(x)


# Decoder
x = layers.Conv2DTranspose(8, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(16, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["mse", psnr, ssim]
)

history1 = autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(valid_data, valid_data),
)
scores1 = autoencoder.evaluate(test_data, test_data)

history2 = autoencoder.fit(
    x=noisy_train_data,
    y=train_data,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(noisy_test_data, test_data),
)
scores2 = autoencoder.evaluate(noisy_test_data, test_data)

print(
    history1.history,
    scores1,
    history2.history,
    scores2,
    sep="\n",
    file=open("data/actoencoder-histories-JNB%s-%d.txt" % (q, trial), "w"),
)

2024-06-12 00:03:35.386811: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


313/313 [==============================] - 2s 5ms/step - loss: 0.6022 - mse: 0.1254 - psnr: 9.1730 - ssim: 0.0914
